# Samuel Watkins, 3032132676

# HW 5: Database
## Weather Prediction, due Monday Mar 12, 2018 at 2 PM

In this assignment, we will create a database to analyze historical weather data and discover the relationships between major cities.

1. Find a list of the 50 most travelled airports in the US and make a table containing the relevant information, such as name, nearest city, latitude, and longitude.
2. Build another table that will hold historical weather information, such as min/max temperature, humidity, and precipitation.
3. Build a web crawler that will pull historical data from Weather Underground from 2008 until now and populate your tables accordingly.
4. For each pair of cities/airports, determine how the daily change of temperature high and precipitation from one city predicts the daily change of the other city 1, 3, and 7 days in advance.
5. Plot the correlation strengths for the 10 top pairs for all three dates for temperature and precipitation (separately) as a function of distance. Also, make a plot as a function of longitude. What trends do you see?

# 1. List of 50 Most Travelled Airports

A list of all airports with latitude, longitude, altitude, codes, and more was found at [this link](). A CSV can be downloaded that contains all of the information, called `airports.dat`.  
A list of the 50 most travelled airports in the US was found at [this link](https://www.rita.dot.gov/bts/sites/rita.dot.gov.bts/files/publications/national_transportation_statistics/html/table_01_44.html/). A CSV can be downloaded called `table_01_44_5.csv`, which contains the airport name, closest city, and the airport code.

In [46]:
import pandas as pd

# read the CSV containing the codes of the 50 most travelled airports
most_travelled_airports = pd.read_csv('table_01_44_5.csv',sep=',',skiprows=4,
                                      usecols=[1],names=["IATA Code"],skipfooter=11,engine="python")

# read the CSV containing information on all airports
all_airports = pd.read_csv('airports.dat',sep=',',usecols=[1,2,3,4,5,6,7,8],
                           names=["Name","City","Country","IATA Code","ICAO Code","Latitude","Longitude","Altitude"])

# merge the CSVs such that we have all relevant information necessary on the 50 most travelled airports
most_travelled_airports = most_travelled_airports.merge(all_airports,how="left",on="IATA Code")

In [61]:
import sqlite3

# let's create an airports table to store all of the information
connection = sqlite3.connect("/tmp/example2.db")
cursor = connection.cursor()

sql_cmd = """CREATE TABLE airports (aid INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT, city TEXT, country TEXT, iata TEXT, icao TEXT, latitude FLOAT, longitude FLOAT, altitude FLOAT)"""

cursor.execute(sql_cmd)

# now populate the table
for ii,row in most_travelled_airports.iterrows():
    name = row["Name"].replace("'","`")
    city = row["City"]
    country = row["Country"]
    icao = row["ICAO Code"]
    iata = row["IATA Code"]
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    altitude = row["Altitude"]
    
    sql_cmd = "INSERT INTO airports" + \
              "(name, city, country, iata, icao, latitude, longitude, altitude)" + \
              f""" VALUES ('{name}','{city}','{country}','{iata}','{icao}',{latitude},{longitude},{altitude})"""
    cursor.execute(sql_cmd)
connection.commit()


# 2. Table for Storing Weather Information